In [78]:
import openai
import os
import json
import re

In [28]:
from openai import OpenAI

In [29]:
client = OpenAI(api_key='xxx') #Add your API key here

In [4]:
def read_text_files(folder_path):
    text_files_content = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            path = os.path.join(folder_path, filename)
            with open(path, 'r', encoding='utf-8') as file:
                text_files_content[filename] = file.read()
    return text_files_content

In [8]:
def synthesize_text_with_chat_gpt(text):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Tu es un modèle de langue spécialisé dans la rédaction de résumés de fragments de texte. Le texte suivant est un extrait d'une œuvre de fiction. Lisez-le et résumez son idée principale en une seule phrase concise :"},
            {"role": "user", "content": text}
        ]
    )
    return completion.choices[0].message

In [6]:
folder_path = 'xxx'
texts = read_text_files(folder_path)
syntheses = []

In [9]:
for filename, content in texts.items():
    one_line_synthesis = synthesize_text_with_chat_gpt(content)
    syntheses.append({filename: one_line_synthesis})

print("Synthesis complete.")

Synthesis complete.


In [ ]:
syntheses

In [15]:
serializable_syntheses = []

for synthesis in syntheses:
    for filename, completion_message in synthesis.items():
        text_content = completion_message.content
        serializable_syntheses.append({filename: text_content})

In [16]:
with open('syntheses_serializable.json', 'w', encoding='utf-8') as json_file:
    json.dump(serializable_syntheses, json_file, ensure_ascii=False, indent=4)

print("Serialized output saved to syntheses_serializable.json.")

Serialized output saved to syntheses_serializable.json.


In [18]:
serializable_syntheses_sorted = sorted(serializable_syntheses, key=lambda x: list(x.keys())[0])

In [19]:
with open('syntheses_serializable_sorted.json', 'w', encoding='utf-8') as json_file:
    json.dump(serializable_syntheses_sorted, json_file, ensure_ascii=False, indent=4)

print("Sorted serialized output saved to syntheses_serializable_sorted.json.")


Sorted serialized output saved to syntheses_serializable_sorted.json.


In [36]:
folder_path = 'xxx'

In [68]:
with open('syntheses_serializable_sorted.json', 'r', encoding='utf-8') as json_file:
    serializable_syntheses_sorted = json.load(json_file)

In [90]:
def extract_numerical_order(filename):
    match = re.search(r'chunk_(\d+)', filename)
    if match:
        return int(match.group(1))
    return -1

In [92]:
def extract_work_identifier(filename):
    return filename.split("_")[0]

def create_batches_with_identifiers(syntheses, batch_size=10):
    batches = []
    current_batch = []
    current_work_id = None

    for synthesis in syntheses:
        filename = list(synthesis.keys())[0]
        work_id = extract_work_identifier(filename)
        
        if current_work_id is None:
            current_work_id = work_id
        
        if work_id != current_work_id or len(current_batch) == batch_size:
            if current_batch:
                batches.append((current_work_id, current_batch))
            current_batch = []
            current_work_id = work_id

        current_batch.append(synthesis)
    
    # Add the last batch
    if current_batch:
        batches.append((current_work_id, current_batch))
    
    return batches

In [93]:
def concatenate_texts(batch):
    concatenated_texts = ""
    for synthesis in batch:
        text = list(synthesis.values())[0]
        concatenated_texts += text + " "
    return concatenated_texts.strip()

In [105]:
def resynthesizer(text):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "u es un modèle de langue spécialisé dans la rédaction de résumés de fragments de texte. Tu liras quelques résumés de divers extraits d'une œuvre de fiction. Lis-les et résumez leurs idées principales en une seule phrase concise."},
            {"role": "user", "content": text}
        ],
        max_tokens=100
    )
    return completion.choices[0].message.content

In [106]:
def resynthesize_batches_with_identifiers(batches_with_identifiers):
    high_level_syntheses = {}
    for work_id, batch in batches_with_identifiers:
        if work_id not in high_level_syntheses:
            high_level_syntheses[work_id] = []
            
        # Perform synthesis for the current batch
        high_level_synthesis = resynthesizer(concatenate_texts(batch))
        
        # Store the synthesis in the list under the current work ID
        high_level_syntheses[work_id].append(high_level_synthesis)
        
        print(f"Work ID: {work_id}, Synthesis: {high_level_synthesis}\n")
        
    return high_level_syntheses

In [107]:
serializable_syntheses_sorted = sorted(serializable_syntheses_sorted, key=lambda x: (list(x.keys())[0].split('_')[0], extract_numerical_order(list(x.keys())[0])))

In [108]:
serializable_syntheses_sorted

[{"1855_Barbara-Charles_L'assassinat-du-pont-rouge.txt_chunk_0.txt": "Deux amis discutent de la douleur et de la souffrance comme sources d'inspiration pour l'art, tandis que l'un fait face à la misère avec patience et détermination, contrastant avec l'autre, plus préoccupé par les obstacles de la vie d'artiste."},
 {"1855_Barbara-Charles_L'assassinat-du-pont-rouge.txt_chunk_1.txt": "Un homme nommé Max apprend, grâce à son ami Rodolphe, des détails troublants sur une femme élégante et talentueuse qu'il connaît, dont le mari, un ancien agent de change, s'est suicidé après avoir détourné des fonds importants, plongeant ainsi leur famille dans la misère."},
 {"1855_Barbara-Charles_L'assassinat-du-pont-rouge.txt_chunk_2.txt": "Un homme nommé Max se retrouve avec son ami Henri de Villiers dans le jardin du Luxembourg, où ils rencontrent Clément, un individu au visage cadavéreux et à l'apparence sordide, suscitant des impressions pénibles et mystérieuses, ce qui les pousse à se demander quel

In [109]:
batches_with_identifiers = create_batches_with_identifiers(serializable_syntheses_sorted, batch_size=10)

In [110]:
batches_with_identifiers

[('1855',
  [{"1855_Barbara-Charles_L'assassinat-du-pont-rouge.txt_chunk_0.txt": "Deux amis discutent de la douleur et de la souffrance comme sources d'inspiration pour l'art, tandis que l'un fait face à la misère avec patience et détermination, contrastant avec l'autre, plus préoccupé par les obstacles de la vie d'artiste."},
   {"1855_Barbara-Charles_L'assassinat-du-pont-rouge.txt_chunk_1.txt": "Un homme nommé Max apprend, grâce à son ami Rodolphe, des détails troublants sur une femme élégante et talentueuse qu'il connaît, dont le mari, un ancien agent de change, s'est suicidé après avoir détourné des fonds importants, plongeant ainsi leur famille dans la misère."},
   {"1855_Barbara-Charles_L'assassinat-du-pont-rouge.txt_chunk_2.txt": "Un homme nommé Max se retrouve avec son ami Henri de Villiers dans le jardin du Luxembourg, où ils rencontrent Clément, un individu au visage cadavéreux et à l'apparence sordide, suscitant des impressions pénibles et mystérieuses, ce qui les pousse à 

In [111]:
high_level_syntheses = resynthesize_batches_with_identifiers(batches_with_identifiers)

Work ID: 1855, Synthesis: Un groupe d'amis discute des aspects troublants et mystérieux de la vie, mettant en lumière les luttes intérieures, les revers financiers et les transformations personnelles, tout en explorant les thèmes de la douleur, de la souffrance et de l'inspiration artistique.

Work ID: 1855, Synthesis: Clément, un homme en difficulté financière malgré ses succès professionnels, trouve un soutien inattendu auprès d'un prêtre et décide de jouer une audacieuse comédie pour améliorer sa situation malgré le supplice moral de sa double vie, suscitant des questionnements sur la moralité et la souffrance entre lui et son ami Max.

Work ID: 1855, Synthesis: Rosalie et Clément sont confrontés à des tensions et mystères autour de leur relation, notamment concernant la présence de Destroy dans la vie de Rosalie, les leçons de musique, l'identité de Mme Thillard-Ducornet, les questionnements existentiels sur la religion, les troubles et réactions suscités par des soupçons sur leur 

In [112]:
with open('syntheses_by_chunk.json', 'w', encoding='utf-8') as json_file:
    json.dump(serializable_syntheses_sorted, json_file, ensure_ascii=False, indent=4)

In [113]:
with open('syntheses_batches_of_10.json', 'w', encoding='utf-8') as json_file:
    json.dump(high_level_syntheses, json_file, ensure_ascii=False, indent=4)